In [1]:
import requests
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
import plotly.express as px
import numpy as np
import json
import math
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
#Get options to
## 1. validate existence of selected vars
## 2. retrieve datatypes and labels for later use
r = requests.options('http://127.0.0.1:8000/assessment?hierarchical=False&auto=False')
options_dict = r.json()

In [3]:
url = 'http://127.0.0.1:8000/assessment'
r = requests.get(url)
j = r.text
df = pd.read_json(j)

In [4]:
colvars=[
'nation__name',
'embarkation_region__export_area__name',
'disembarkation_region__name',
'disembarkation_region__import_area__name']

rowvars=[
'year',
'nation__name',
'embarkation_region__export_area__name',
'disembarkation_region__name',
'disembarkation_region__import_area__name'
]

cellvars=[
'embarked_slaves',
'disembarked_slaves'
]

timevars=['year']

In [5]:
colnamemap={i:options_dict[i]['label'] for i in options_dict}
df=pd.read_json(r.text)
df_formatted=df.rename(columns=colnamemap)
df_formatted

,ID,Enslaving Nation: ID,Enslaving Nation,Enslaving Nation: Order num,Embarkation Region: ID,Embarkation Region: ID,Broad Embarkation Region,Embarkation Area: Order num,Embarkation Area: Latitude,Embarkation Area: Longitude,...,Broad Disembarkation Region: Show on map,Specific Disembarkation Region,Specific Disembarkation Region: Order num,Specific Disembarkation Region: Latitude,Specific Disembarkation Region: Longitude,Specific Disembarkation Region: Show at zoom,Specific Disembarkation Region: Show on map,Year,Embarked slaves,Disembarked slaves
0,1,1,Spain / Uruguay,1,1,1,Africa,1,0.054033,9.343897,...,True,Other Spanish Americas,30,18.466700,-69.900000,999,True,1501,95.000000,67.000000
1,2,1,Spain / Uruguay,1,1,1,Africa,1,0.054033,9.343897,...,True,Other Spanish Americas,30,18.466700,-69.900000,999,True,1502,95.000000,67.000000
2,3,1,Spain / Uruguay,1,1,1,Africa,1,0.054033,9.343897,...,True,Other Spanish Americas,30,18.466700,-69.900000,999,True,1503,95.000000,67.000000
3,4,1,Spain / Uruguay,1,1,1,Africa,1,0.054033,9.343897,...,True,Other Spanish Americas,30,18.466700,-69.900000,999,True,1504,95.000000,67.000000
4,5,1,Spain / Uruguay,1,1,1,Africa,1,0.054033,9.343897,...,True,Other Spanish Americas,30,18.466700,-69.900000,999,True,1505,95.000000,67.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9326,9327,7,Denmark / Baltic,7,2,1,Africa,1,0.054033,9.343897,...,True,Danish West Indies,25,18.200000,-64.550000,999,True,1805,840.430407,729.638258
9327,9328,7,Denmark / Baltic,7,4,1,Africa,1,0.054033,9.343897,...,True,Jamaica,7,17.881263,-76.522778,999,True,1806,577.939210,532.764008
9328,9329,7,Denmark / Baltic,7,4,1,Africa,1,0.054033,9.343897,...,True,Danish West Indies,25,18.200000,-64.550000,999,True,1806,335.577606,304.816026
9329,9330,7,Denmark / Baltic,7,4,1,Africa,1,0.054033,9.343897,...,True,Rio de la Plata,29,-34.587500,-58.672500,999,True,1806,562.685682,501.117826


In [6]:
colvar_options = [options_dict[i]['label'] for i in colvars]

colvar_emb = widgets.Dropdown(
    options=colvar_options,
    description="Columns",
    disabled=False,
)

In [7]:
rowvar_options = [options_dict[i]['label'] for i in rowvars]

rowvar_emb = widgets.Dropdown(
    options=rowvar_options,
    description="Rows",
    disabled=False,
)

In [8]:
cellvar_options = [options_dict[i]['label'] for i in cellvars]

cellvar_emb = widgets.SelectMultiple(
    options=cellvar_options,
    value=cellvar_options,
    description="Cells",
    disabled=False
)

In [23]:
dropnavar_emb = widgets.Checkbox(
    value=False,
    description="Drop Empty?",
    disabled=False
)

In [24]:
years=df_formatted['Year']
minyear,maxyear=[years.min(),years.max()]
year_range_emb=widgets.IntRangeSlider(value=[minyear,maxyear],min=minyear,max=maxyear,description="Years:")

In [39]:
year_range_emb

IntRangeSlider(value=(1501, 1766), description='Years:', max=1866, min=1501)

In [40]:
enslaving_nations_varlabel=options_dict['nation__name']['label']
enslaving_nations=list(df_formatted[enslaving_nations_varlabel].unique())

enslaving_nations_emb = widgets.SelectMultiple(
    options=enslaving_nations,
    value=enslaving_nations,
    description="Cells",
    disabled=False
)


In [42]:
enslaving_nations_emb

SelectMultiple(description='Cells', index=(0, 1), options=('Spain / Uruguay', 'Portugal / Brazil', 'Great Brit…

In [43]:
enslaving_nations_emb.value

('Spain / Uruguay', 'Portugal / Brazil')

In [44]:
widgets.TwoByTwoLayout(top_left=rowvar_emb,top_right=colvar_emb,bottom_left=dropnavar_emb,bottom_right=cellvar_emb)

TwoByTwoLayout(children=(Dropdown(description='Rows', layout=Layout(grid_area='top-left'), options=('Year', 'E…

In [45]:
rowvar=rowvar_emb.value
colvar=colvar_emb.value
cellvar=cellvar_emb.value

In [46]:
years=df_formatted['Year']
max_epoch_length=year_range_emb.value[1]-year_range_emb.value[0]
epoch_length_emb=widgets.IntSlider(value=max_epoch_length,min=1,max=max_epoch_length,description="Epoch Length:")

In [51]:
if 'Year' in rowvar:
    display(epoch_length_emb)

IntSlider(value=50, description='Epoch Length:', max=265, min=1)

In [52]:
selected_vars=list(set(list(cellvar_emb.value)+[colvar_emb.value]+[rowvar_emb.value]+['Year']))
df_selected=df_formatted[selected_vars]
df_selected = df_selected[df_selected['Year'].between(year_range_emb.value[0],year_range_emb.value[1])]
df_selected = df_selected[df_selected[enslaving_nations_varlabel].isin(list(enslaving_nations_emb.value))]

In [53]:
if rowvar=='Year':
    bins=math.ceil((year_range_emb.value[1]-year_range_emb.value[0])/epoch_length_emb.value)
    if bins!=1:
        df_selected['Year']=pd.cut(df_selected['Year'],bins)

In [54]:
pd.pivot_table(df_selected,values=cellvar_emb.value,index=rowvar_emb.value,columns=colvar_emb.value,dropna=dropnavar_emb.value,aggfunc=np.sum)

Disembarked slaves                   Embarked slaves  \
Enslaving Nation      Portugal / Brazil Spain / Uruguay Portugal / Brazil   
Year                                                                        
(1500.735, 1545.167]       19382.573034    18925.534000      27669.958619   
(1545.167, 1589.333]       53243.044269    42345.940500      73428.920385   
(1589.333, 1633.5]        328571.812044    93280.206022     409868.456361   
(1633.5, 1677.667]        319474.715000    29566.800000     377312.037074   
(1677.667, 1721.833]      591921.867000     3863.000000     668701.960870   
(1721.833, 1766.0]        853059.241900      958.000000     964277.428415   

                                      
Enslaving Nation     Spain / Uruguay  
Year                                  
(1500.735, 1545.167]    27025.761429  
(1545.167, 1589.333]    60488.915000  
(1589.333, 1633.5]     133265.151461  
(1633.5, 1677.667]      40551.714286  
(1677.667, 1721.833]     4900.000000  
(1721.833, 1766.0]       1106.000000

In [56]:
#Selects only the first of the 1-2 numeric vars
df_selected["Year"]=df_selected["Year"].astype('str')
fig = px.bar(df_selected, x="Year",
             y=cellvar[0], color=colvar
             )
fig.show()